# Enterprise Steam

This notebook is intended to help you get started managing your AI Engines in the H2O AI Cloud using python. 

* **Product Documentation:** https://docs.h2o.ai/steam/index.html
* **Python Documentation:** https://docs.h2o.ai/enterprise-steam/latest-stable/docs/python-docs/index.html

## Prerequisites 
This tutorial relies on the latest Steam SDK (1.8.11) which can be installed into a python environment by: 

1. Click on `My AI Engines` from the H2O AI Cloud and then `Python client` to download the wheel file
3. Navigate to the location where the python client was downloaded and install the client using `pip install h2osteam-1.8.11-py2.py3-none-any.whl`

We also require the `h2o_authn` library for securely connecting to the H2O AI Cloud platform: `pip install h2o_authn`.

We also set the following variables to connect to a specific H2O AI Cloud environment. They can be found by logging into the platform, clicking on your name, and choosing the `CLI & API Access` page. Then, copy values from the `Accessing H2O AI Cloud APIs` section.

In [1]:
CLIENT_ID = "q8s-internal-platform"
TOKEN_ENDPOINT = "https://auth.demo.h2o.ai/auth/realms/q8s-internal/protocol/openid-connect/token"
REFRESH_TOKEN = "https://cloud-internal.h2o.ai/auth/get-platform-token"

H2O_STEAM_URL = "https://steam.cloud-internal.h2o.ai/"

In [2]:
from getpass import getpass

import h2o_authn
import h2osteam
from h2osteam.clients import DriverlessClient
from h2osteam.clients import H2oKubernetesClient

## Securely connect

We first connect to the H2O AI Cloud using our personal access token to create a token provider object. We can then use this object to log into Steam and other APIs.

In [3]:
print(f"Visit {REFRESH_TOKEN} to get your personal access token")
tp = h2o_authn.TokenProvider(
    refresh_token=getpass("Enter your access token: "),
    client_id=CLIENT_ID,
    token_endpoint_url=TOKEN_ENDPOINT
)

Visit https://cloud-internal.h2o.ai/auth/get-platform-token to get your personal access token
Enter your access token: ········


Then connect to Steam to start managing your AI Engines.

In [4]:
steam = h2osteam.login(
    url=H2O_STEAM_URL,
    access_token=tp()
)

## Understanding your environment

* Ensure that your python client and backend versions match. 
* View what AI Engines and environments are available 

In [5]:
print(f"Backend Version: {h2osteam.api().get_config_meta()['version']}")
print(f"Library Version: {h2osteam.__version__}")

Backend Version: 1.8.11
Library Version: 1.8.11


In [6]:
h2osteam.api().get_config_meta()

{'version': '1.8.11',
 'build': '1.8.10-30-gaf0c13e-dirty',
 'built': 'Thu Feb  3 00:05:06 UTC 2022',
 'restart_pending': False,
 'support_email': 'cloud-feedback@h2o.ai',
 'license_valid': True,
 'is_hadoop_enabled': False,
 'is_kubernetes_enabled': True,
 'is_h2o_enabled': True,
 'is_h2o_running': True,
 'is_sparkling_enabled': False,
 'is_sparkling_running': False,
 'is_driverless_enabled': True,
 'is_driverless_running': True,
 'is_h2o_engine_uploaded': False,
 'is_h2o_kubernetes_engine_uploaded': True,
 'is_sparkling_engine_uploaded': False,
 'is_driverless_engine_uploaded': True,
 'driverless_backend_type': 'kubernetes',
 'is_minio_enabled': False,
 'h2o_backend_type': 'kubernetes',
 'inside_cluster': True}

## Managing Driverless AI

### View available versions

In [7]:
for dai_version in steam.get_driverless_engines():
    print(dai_version["version"])

1.9.0.6
1.9.1.1
1.9.1.3
1.9.2.1
1.9.3
1.10.0
1.10.1
1.10.1.1
1.10.1.2
1.10.1.3
1.10.2


### View available profiles

* List the profiles
* View the options for max run time in this profile

In [8]:
for profile in steam.get_profiles():
    if profile["profile_type"] == "driverless_kubernetes":
        print(profile["name"])

default-driverless-kubernetes
se-driverless-kubernetes


In [9]:
steam.get_profile_by_name("default-driverless-kubernetes")["driverless_kubernetes"]["max_uptime_hours"]

{'id': 835,
 'min': 1,
 'is_min_set': True,
 'max': 24,
 'is_max_set': True,
 'initial': 8,
 'is_initial_set': True,
 'profile_max': -1,
 'is_profile_max_set': True}

### Create an instance
See the Driverless AI Tutorial for what to do once you've connected to the AutoML platform. 

In [10]:
dai = DriverlessClient().launch_instance(
    name="test-instance",
    version="1.10.2",
    profile_name="default-driverless-kubernetes",
    max_uptime_h=1
)
client = dai.connect()

Driverless AI instance is submitted, please wait...
Driverless AI instance is running


In [11]:
dai.details()

{'id': 482,
 'profile_name': 'default-driverless-kubernetes',
 'name': 'test-instance',
 'status': 'running',
 'target_status': 'running',
 'version': '1.10.2',
 'backend_type': '',
 'instance_type': 'singlenode',
 'master_id': -1,
 'cpu_count': 8,
 'gpu_count': 0,
 'memory_gb': 30,
 'storage_gb': 64,
 'max_idle_seconds': 7200,
 'max_uptime_seconds': 3600,
 'timeout_seconds': 1800,
 'address': 'http://10.3.223.206:12345',
 'authentication': 'oidc',
 'password': '',
 'created_at': 1649889164,
 'started_at': 1649889198,
 'created_by': 'michelle.tanco@h2o.ai',
 'current_uptime_seconds': 12,
 'current_idle_seconds': 12,
 'pod_latest_event': None,
 'service_latest_event': None,
 'pvc_latest_event': None,
 'stop_reason': '',
 'config_toml': '',
 'volumes': ''}

In [12]:
client.server.gui()

'https://steam.cloud-internal.h2o.ai:443/proxy/driverless/482'

### List my instances

In [13]:
for instance in steam.get_driverless_instances():
    print(instance["id"], "\t", instance["profile_name"], "\t", instance["status"], "\t", instance["name"])

74 	 default-driverless-kubernetes 	 stopped 	 mt-demo
445 	 default-driverless-kubernetes 	 stopped 	 MT-test
446 	 default-driverless-kubernetes 	 stopped 	 dai-quickstart
482 	 default-driverless-kubernetes 	 running 	 test-instance


### Download instance logs

In [14]:
dai = DriverlessClient().get_instance(name="test-instance")

In [15]:
dai.download_logs('dai_test_instance_logs.log')

Driverless AI instance logs saved to dai_test_instance_logs.log


### Pause an instance

You can pause an instance that is currently running. Pausing an instance shuts it down to save resources, it is similar to powering off a server. You will not loose any data and you can start an instance at any time.

In [16]:
dai.stop()

Driverless AI instance is stopping, please wait...
Driverless AI instance is stopped


### Resume the instance

You can resume a paused instance by running:

In [17]:
dai.start()

Driverless AI instance is starting, please wait...
Driverless AI instance is running


### Delete the instance

When you no longer need an instance, you can terminate it. Once deleted, there is no way to restart the instance or access any data.

In [18]:
dai.terminate()

In [19]:
# it takes ~20 seconds for the status to update
for instance in steam.get_driverless_instances():
    print(instance["id"], "\t", instance["profile_name"], "\t", instance["status"], "\t", instance["name"])

74 	 default-driverless-kubernetes 	 stopped 	 mt-demo
445 	 default-driverless-kubernetes 	 stopped 	 MT-test
446 	 default-driverless-kubernetes 	 stopped 	 dai-quickstart
482 	 default-driverless-kubernetes 	 running 	 test-instance


## Managing H2O-3

### View available versions

In [20]:
for h2o_version in steam.get_h2o_kubernetes_engines():
    print(h2o_version["version"])

3.32.0.5
3.34.0.3
3.34.0.7
3.36.0.1
3.36.0.3


### View available profiles

In [21]:
for profile in steam.get_profiles():
    if profile["profile_type"] == "h2o_kubernetes":
        print(profile["name"])

default-h2o-kubernetes


In [22]:
steam.get_profile_by_name("default-h2o-kubernetes")["h2o_kubernetes"]["max_uptime_hours"]

{'id': 856,
 'min': 1,
 'is_min_set': True,
 'max': 24,
 'is_max_set': True,
 'initial': 12,
 'is_initial_set': True,
 'profile_max': -1,
 'is_profile_max_set': True}

### Create a cluster
See the H2O-3 Tutorial for what to do once you've connected to the distributed ML platform. 

In [23]:
h2o = H2oKubernetesClient().launch_cluster(
    name="test-instance",
    version="3.36.0.3",
    profile_name="default-h2o-kubernetes",
    max_uptime_h=1
)

In [24]:
while not h2o.is_running():
    pass

In [25]:
client = h2o.connect()

Connecting to H2O server at https://steam.cloud-internal.h2o.ai:443/proxy/h2o-k8s/159 ... successful.


H2O_cluster_uptime:,26 secs
H2O_cluster_timezone:,Etc/GMT
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.3
H2O_cluster_version_age:,1 month and 28 days
H2O_cluster_name:,test-instance
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.228 Gb
H2O_cluster_total_cores:,1
H2O_cluster_allowed_cores:,1
H2O_cluster_status:,"locked, healthy"


In [26]:
print("https://" + h2o.get_connection_config()["connect_params"]["ip"] + "/" + h2o.get_connection_config()["connect_params"]["context_path"] + "/flow/index.html")

https://steam.cloud-internal.h2o.ai/proxy/h2o-k8s/159/flow/index.html


### List my clusters

In [27]:
for instance in steam.get_h2o_kubernetes_clusters():
    print(instance["id"], "\t", instance["profile_name"], "\t", instance["status"], "\t", instance["name"])

82 	 default-h2o-kubernetes 	 stopped 	 es-test-instance
83 	 default-h2o-kubernetes 	 stopped 	 es-test-instance-2
159 	 default-h2o-kubernetes 	 running 	 test-instance


### Download cluster logs

In [28]:
h2o = H2oKubernetesClient().get_cluster(name="test-instance", created_by="michelle.tanco@h2o.ai")

In [29]:
h2o.download_logs('h2o_test_instance_logs.log')

H2O cluster logs saved to h2o_test_instance_logs.log


### Stop an instance
H2O clusters cannot be resumed after they are stopped. Only turn off your cluster once you are permanately done with it.

In [30]:
h2o.stop()

In [31]:
print(f"Current status: {h2o.status}")
print(f"Target status: {h2o.target_status}")

Current status: running
Target status: stopped


### Delete an instance

In [32]:
h2o.terminate()

In [33]:
for instance in steam.get_h2o_kubernetes_clusters():
    print(instance["id"], "\t", instance["profile_name"], "\t", instance["status"], "\t", instance["name"])

82 	 default-h2o-kubernetes 	 stopped 	 es-test-instance
83 	 default-h2o-kubernetes 	 stopped 	 es-test-instance-2
